In [ ]:
# Import Libraries

import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pip install transformers

     |████████████████████████████████| 3.8 MB 14.1 MB/s 
     |████████████████████████████████| 895 kB 41.3 MB/s 
     |████████████████████████████████| 6.5 MB 62.1 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip3 install pickle5


     |████████████████████████████████| 256 kB 14.5 MB/s 


1. Data Preparation

In [ ]:
df= np.load('/content/drive/MyDrive/DC-2/embeddings.npy') #pooler_outputs for all the samples.


In [ ]:
import pickle5 as p
with open("/content/drive/MyDrive/DC-2/train.pickle", "rb") as f:
    object = p.load(f)
    
ts= pd.DataFrame.from_dict(object, orient='index')
ts.head()

# Training Set Loaded

In [ ]:
with open("/content/drive/MyDrive/DC-2/test.pickle", "rb") as f:
    object = p.load(f)
    
tst= pd.DataFrame.from_dict(object, orient='index')
tst.head()

# Test Set Loaded

,0
0,"[The battery had gone out again., He could not..."
1,[I enjoyed riding the train to the ski area an...
2,"[We were out working one day cleaning it up., ..."
3,[We set out for a mountain that was close to o...
4,"[She took her toddler outside to play., So Mar..."


2. Tokenization of Data:

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tk=[]

for i in range(df.shape[0]):
      tk.append(tokenizer(ts[0].loc[i], padding='max_length', max_length= 20, truncation=True, return_tensors='tf'))  


# Tokens for the training set are received.

In [ ]:
tstt=[]

for i in range(tst.shape[0]):
      tstt.append(tokenizer(tst[0].loc[i], padding='max_length', max_length= 20, truncation=True, return_tensors='tf'))  

# Tokens for the test set are received.

3. Receiving Embeddings:

In [ ]:
from transformers import TFBertModel

model = TFBertModel.from_pretrained("bert-base-v2")
output= []

for i in range(df.shape[0]):
    
    output.append(model(tk[i]))

output= output['pooler_outputs']

    # Pooler Output Embeddings from the training Set are received and stored in 'output'

In [ ]:
output2= []

for i in range(tst.shape[0]):
    
    output2.append(model(tstt[i]))

  
  output= output['pooler_outputs']

    # Pooler Output Embeddings from the test Set are received and stored in 'output2'

4. Splitting Data and Building Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(output, ts[1].loc[:98031], test_size=0.2, random_state=42)



In [ ]:
num_classes= 5
batch_size= 128
epochs= 25

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model1 = Sequential()
model1.add(Input([5,768], dtype=tf.float32))
model1.add(Flatten())
model1.add(Dense(256, activation = 'relu'))
model1.add(Dense(64, activation = 'relu'))
model1.add(Dense(64, activation = 'relu'))
model1.add(Dense(128, activation = 'relu'))

model1.add(Dense(128, activation = 'relu'))
model1.add(Dense(128, activation = 'relu'))
model1.add(Dense(128, activation = 'relu'))

model1.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['categorical_accuracy'])

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3840)              0         
                                                                 
 dense (Dense)               (None, 256)               983296    
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 128)               1

In [ ]:
model1.fit(np.array([i['pooler_output'] for i in X_train]), np.array(y_train.to_list())/10, validation_data=[np.array([i['pooler_output'] for i in X_test]), np.array(y_test.to_list())/10], epochs= epochs)


5. Testing Data on Test Set:

In [ ]:
pred= model1.predict(output2)

#Predictions are made by feedings in 'output2'

In [ ]:
pred= np.int_(np.round(pred*10))

In [ ]:
pre= pd.DataFrame(pred, columns=['index_1', 'index_2', 'index_3', 'index_4', 'index_5'])

In [ ]:
pre.to_csv('pre2.csv', index_label='id')